In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from flame_net.FourierOp_Nd import FourierOp_Nd
from flame_net.DeepONet_1d import DeepONet_1d
from flame_net.ConvPDE_Nd import ConvPDE_Nd
from flame_net.lib_uti import Cdata_sys, count_learnable_params, lib_Model,lib_DataGen,LpLoss,lib_ModelTrain
from timeit import default_timer

#-----------------------
device = torch.device('cuda') # if torch.cuda.is_available() else 'cpu')


#-----------------------
nDIM = 1
data_sys = Cdata_sys('siva',[0.02],1)
#data_sys = Cdata_sys('cfd',['L1536_rho8'])
params = lib_Model.set_default_params(data_sys,nDIM)

params['T_in' ] = 1
params['T_out'] = 20
params['data_channel'] = 1
#params['data:yB_estimate']=np.array([-0.7, 1.3]) * np.pi
#params['data:yB_estimate']=np.array([-0.8, 2.4]) * np.pi
#params['data:yB_estimate']=np.array([-0.7, 2.5]) * np.pi
#params['data:AspectRatio_set'] = 1 # (2.5+0.7)/2.
params['data:nStep']  = 1
params['data:nStepSkip']=1
params['Nx']            = 512 # 128
#-----
params['train:batch_size'] = 1000
params['train:learning_rate'] = 0.0025
#-----------
model_name = 'fourier'
params['fourier:modes_fourier' ] =  128 #64 #,64]
params['fourier:width' ] =  30 # 20
params['fourier:depth' ] = 4
#model_name = 'conv'
#params['conv:en1_channels' ] = [16,32,64,64,128,128,256,256]
#-----
#model_name = 'onet'
#params['onet:type_branch']='conv'
#params['onet:type_branch']='fc'
##params['onet:P'] = 50
#params['onet:trunk_featurepair']=2
#params['onet:type_trunk']='simple'
#---------



train_disp, test_disp, train_PDEpara, test_PDEpara = lib_DataGen.DataGen(data_sys,params)


----- params for DataGen -----
nDIM,T_in,T_out,Nx,nStep,nStepSkip,data_channel, data_sys.sysname, data_sys.list_nu, data_sys.list_cfdfilename
------------------------------
N= 512 ,dt_Output= 0.015 ,nu_list = [0.02] ,nu_extention_list = [0.02] ,NumOutPut_dt= [37]
ActiveNoise_Amplitude=0.000000,stepfeq=1,k = 0 
[(1, 200001, 0.02, 'rand_FFT_2_8'), (500, 201, 0.02, 'rand_FFT_2_8'), (500, 201, 0.02, 'rand_simple')]
num_traj= 1  leq_seq= 200001  nu= 0.02   rand_FFT_2_8
Sucess: load ./data/dsol_multraj1L200001_nu02_N512_dt015_rand_FFT_2_8.pkl
num_traj= 500  leq_seq= 201  nu= 0.02   rand_FFT_2_8
Sucess: load ./data/dsol_multraj500L201_nu02_N512_dt015_rand_FFT_2_8.pkl
num_traj= 500  leq_seq= 201  nu= 0.02   rand_simple
Sucess: load ./data/dsol_multraj500L201_nu02_N512_dt015_rand_simple.pkl
[(1, 20001, 0.02, 'rand_FFT_2_8'), (50, 201, 0.02, 'rand_FFT_2_8'), (50, 201, 0.02, 'rand_simple')]
num_traj= 1  leq_seq= 20001  nu= 0.02   rand_FFT_2_8
Sucess: load ./data/dsol_multraj1L20001_nu02_N512_dt01

In [2]:
#%matplotlib inline
#plt.figure(figsize=[4,4])
#plt.contourf(  train_disp[-1,:,:,-1]  )

In [3]:
model_name_detail = lib_Model.get_model_name_detail(model_name,data_sys,params)
model = lib_Model.build_model(model_name_detail,params)

Fourier_m128w30nu02_share_skip
count_learnable_params= 235487


In [4]:
params['train:batch_size'] =800
lib_ModelTrain.Train(train_disp, test_disp,train_PDEpara,test_PDEpara,model,model_name_detail,device,params )


batch_size= 800
ntrain= 17998  ,ntest= 1798
train_l2_indentity= tensor(0.2097) ,  test_l2_indentity= tensor(0.2099)
.......................
ep t[s] trainErr_norm testErr_norm train_l2 test_l2 train_l2_step test_l2_step train_rel_ind test_rel_ind 
0, 21.82, 1.000, 0.698, 1.247, 0.871, 1.352, 0.857, 5.947, 4.150
.......................
1, 20.34, 0.547, 0.517, 0.682, 0.644, 0.686, 0.641, 3.253, 3.070
.......................
2, 20.33, 0.461, 0.434, 0.574, 0.541, 0.569, 0.528, 2.739, 2.577
.......................
3, 20.34, 0.469, 0.602, 0.585, 0.751, 0.582, 0.793, 2.791, 3.579
.......................
4, 20.35, 0.452, 0.399, 0.564, 0.498, 0.568, 0.490, 2.690, 2.371
.......................
5, 20.33, 0.406, 0.375, 0.506, 0.467, 0.493, 0.445, 2.415, 2.227
.......................
6, 20.34, 0.312, 0.340, 0.389, 0.424, 0.376, 0.443, 1.856, 2.020
.......................
7, 20.33, 0.321, 0.291, 0.401, 0.363, 0.384, 0.344, 1.911, 1.730
.......................
8, 20.33, 0.256, 0.292, 0.319, 0.364, 0.3

.......................
89, 20.34, 0.082, 0.119, 0.102, 0.148, 0.096, 0.143, 0.486, 0.705
.......................
90, 20.34, 0.086, 0.129, 0.108, 0.160, 0.109, 0.180, 0.513, 0.764
.......................
91, 20.33, 0.079, 0.060, 0.099, 0.075, 0.098, 0.072, 0.471, 0.358
.......................
92, 20.33, 0.069, 0.095, 0.086, 0.119, 0.081, 0.119, 0.409, 0.567
.......................
93, 20.33, 0.077, 0.098, 0.096, 0.123, 0.092, 0.110, 0.458, 0.584
.......................
94, 20.33, 0.073, 0.104, 0.092, 0.130, 0.087, 0.144, 0.437, 0.621
.......................
95, 20.33, 0.089, 0.105, 0.110, 0.131, 0.108, 0.118, 0.527, 0.625
.......................
96, 20.34, 0.083, 0.077, 0.104, 0.096, 0.099, 0.090, 0.495, 0.458
.......................
97, 20.35, 0.069, 0.072, 0.087, 0.090, 0.081, 0.091, 0.413, 0.430
.......................
98, 20.34, 0.080, 0.124, 0.099, 0.155, 0.101, 0.171, 0.474, 0.738
.......................
99, 20.33, 0.077, 0.063, 0.096, 0.079, 0.096, 0.080, 0.460, 0.377
..........

.......................
179, 20.34, 0.064, 0.046, 0.080, 0.057, 0.074, 0.053, 0.381, 0.273
.......................
180, 20.33, 0.043, 0.057, 0.054, 0.071, 0.051, 0.067, 0.256, 0.338
.......................
181, 20.34, 0.043, 0.044, 0.053, 0.055, 0.050, 0.053, 0.255, 0.264
.......................
182, 20.33, 0.039, 0.050, 0.049, 0.062, 0.047, 0.064, 0.235, 0.295
.......................
183, 20.39, 0.042, 0.059, 0.052, 0.073, 0.050, 0.067, 0.248, 0.350
.......................
184, 20.34, 0.046, 0.048, 0.057, 0.060, 0.053, 0.057, 0.274, 0.286
.......................
185, 20.33, 0.042, 0.059, 0.053, 0.074, 0.053, 0.077, 0.252, 0.353
.......................
186, 20.33, 0.052, 0.056, 0.065, 0.069, 0.066, 0.064, 0.309, 0.330
.......................
187, 20.34, 0.046, 0.050, 0.057, 0.063, 0.055, 0.062, 0.273, 0.299
.......................
188, 20.32, 0.047, 0.058, 0.058, 0.073, 0.054, 0.070, 0.277, 0.346
.......................
189, 20.32, 0.051, 0.051, 0.064, 0.064, 0.059, 0.062, 0.305, 0.303

.......................
269, 20.32, 0.029, 0.034, 0.036, 0.042, 0.033, 0.038, 0.171, 0.199
.......................
270, 20.33, 0.034, 0.047, 0.042, 0.059, 0.039, 0.054, 0.201, 0.280
.......................
271, 20.32, 0.033, 0.033, 0.042, 0.041, 0.040, 0.039, 0.199, 0.194
.......................
272, 20.32, 0.028, 0.036, 0.035, 0.045, 0.034, 0.044, 0.166, 0.212
.......................
273, 20.32, 0.026, 0.026, 0.032, 0.032, 0.030, 0.031, 0.152, 0.152
.......................
274, 20.32, 0.029, 0.030, 0.036, 0.038, 0.034, 0.035, 0.171, 0.179
.......................
275, 20.32, 0.026, 0.024, 0.032, 0.030, 0.031, 0.029, 0.155, 0.144
.......................
276, 20.32, 0.027, 0.028, 0.034, 0.035, 0.032, 0.033, 0.161, 0.168
.......................
277, 20.33, 0.025, 0.026, 0.032, 0.033, 0.030, 0.030, 0.150, 0.156
.......................
278, 20.32, 0.029, 0.056, 0.036, 0.069, 0.034, 0.062, 0.171, 0.331
.......................
279, 20.32, 0.035, 0.029, 0.044, 0.036, 0.041, 0.034, 0.211, 0.170

.......................
359, 20.33, 0.021, 0.021, 0.026, 0.026, 0.025, 0.025, 0.123, 0.124
.......................
360, 20.32, 0.019, 0.020, 0.024, 0.025, 0.023, 0.023, 0.116, 0.118
.......................
361, 20.33, 0.023, 0.026, 0.028, 0.032, 0.027, 0.031, 0.135, 0.154
.......................
362, 20.33, 0.022, 0.020, 0.028, 0.025, 0.027, 0.023, 0.133, 0.117
.......................
363, 20.33, 0.020, 0.022, 0.024, 0.028, 0.024, 0.028, 0.117, 0.133
.......................
364, 20.35, 0.020, 0.018, 0.024, 0.023, 0.024, 0.022, 0.117, 0.109
.......................
365, 20.32, 0.019, 0.019, 0.023, 0.024, 0.022, 0.022, 0.112, 0.112
.......................
366, 20.32, 0.019, 0.020, 0.024, 0.025, 0.023, 0.024, 0.115, 0.118
.......................
367, 20.33, 0.019, 0.020, 0.024, 0.025, 0.023, 0.024, 0.114, 0.120
.......................
368, 20.32, 0.021, 0.021, 0.026, 0.026, 0.025, 0.025, 0.123, 0.125
.......................
369, 20.33, 0.019, 0.019, 0.024, 0.023, 0.023, 0.022, 0.115, 0.112

...

.......................
539, 20.32, 0.015, 0.015, 0.019, 0.019, 0.018, 0.018, 0.089, 0.092
.......................
540, 20.33, 0.015, 0.015, 0.018, 0.019, 0.018, 0.018, 0.088, 0.091
.......................
541, 20.32, 0.015, 0.015, 0.019, 0.019, 0.018, 0.018, 0.088, 0.090
.......................
542, 20.33, 0.015, 0.016, 0.019, 0.019, 0.018, 0.019, 0.089, 0.093
.......................
543, 20.34, 0.015, 0.015, 0.019, 0.019, 0.018, 0.018, 0.089, 0.091
.......................
544, 20.33, 0.015, 0.016, 0.019, 0.019, 0.018, 0.018, 0.089, 0.092
.......................
545, 20.32, 0.015, 0.016, 0.018, 0.020, 0.018, 0.019, 0.087, 0.094
.......................
546, 20.32, 0.015, 0.016, 0.019, 0.020, 0.018, 0.019, 0.089, 0.094
.......................
547, 20.33, 0.015, 0.015, 0.019, 0.019, 0.018, 0.018, 0.089, 0.091
.......................
548, 20.32, 0.015, 0.016, 0.019, 0.019, 0.018, 0.019, 0.089, 0.092
.......................
549, 20.33, 0.015, 0.016, 0.018, 0.020, 0.018, 0.019, 0.087, 0.094

.......................
629, 20.33, 0.014, 0.015, 0.018, 0.018, 0.017, 0.017, 0.084, 0.087
.......................
630, 20.33, 0.014, 0.015, 0.018, 0.019, 0.017, 0.018, 0.084, 0.090
.......................
631, 20.33, 0.014, 0.015, 0.018, 0.018, 0.017, 0.017, 0.084, 0.087
.......................
632, 20.33, 0.014, 0.014, 0.017, 0.018, 0.017, 0.017, 0.083, 0.086
.......................
633, 20.33, 0.014, 0.015, 0.017, 0.018, 0.017, 0.017, 0.083, 0.087
.......................
634, 20.37, 0.014, 0.015, 0.017, 0.018, 0.017, 0.017, 0.083, 0.086
.......................
635, 20.33, 0.014, 0.015, 0.018, 0.018, 0.017, 0.017, 0.084, 0.086
.......................
636, 20.33, 0.014, 0.015, 0.017, 0.018, 0.017, 0.017, 0.083, 0.087
.......................
637, 20.34, 0.014, 0.015, 0.018, 0.018, 0.017, 0.017, 0.084, 0.087
.......................
638, 20.33, 0.014, 0.015, 0.018, 0.018, 0.017, 0.017, 0.084, 0.087
.......................
639, 20.33, 0.014, 0.015, 0.018, 0.019, 0.017, 0.018, 0.084, 0.088

.......................
719, 20.27, 0.014, 0.014, 0.017, 0.018, 0.017, 0.017, 0.081, 0.084
.......................
720, 20.27, 0.014, 0.014, 0.017, 0.018, 0.016, 0.017, 0.081, 0.084
.......................
721, 20.27, 0.014, 0.014, 0.017, 0.018, 0.017, 0.017, 0.081, 0.085
.......................
722, 20.27, 0.014, 0.014, 0.017, 0.018, 0.017, 0.017, 0.081, 0.084
.......................
723, 20.27, 0.014, 0.014, 0.017, 0.018, 0.016, 0.017, 0.081, 0.085
.......................
724, 20.27, 0.014, 0.014, 0.017, 0.018, 0.017, 0.017, 0.081, 0.086
.......................
725, 20.27, 0.014, 0.014, 0.017, 0.018, 0.017, 0.017, 0.081, 0.084
.......................
726, 20.27, 0.014, 0.014, 0.017, 0.018, 0.017, 0.017, 0.081, 0.084
.......................
727, 20.27, 0.014, 0.014, 0.017, 0.018, 0.017, 0.017, 0.081, 0.084
.......................
728, 20.27, 0.014, 0.014, 0.017, 0.018, 0.017, 0.017, 0.081, 0.085
.......................
729, 20.27, 0.014, 0.014, 0.017, 0.018, 0.016, 0.017, 0.081, 0.084

.......................
809, 20.27, 0.013, 0.014, 0.017, 0.017, 0.016, 0.017, 0.080, 0.083
.......................
810, 20.27, 0.013, 0.014, 0.017, 0.017, 0.016, 0.017, 0.079, 0.083
.......................
811, 20.27, 0.013, 0.014, 0.017, 0.017, 0.016, 0.017, 0.080, 0.083
.......................
812, 20.27, 0.013, 0.014, 0.017, 0.018, 0.016, 0.017, 0.080, 0.084
.......................
813, 20.27, 0.013, 0.014, 0.017, 0.017, 0.016, 0.017, 0.080, 0.083
.......................
814, 20.27, 0.013, 0.014, 0.017, 0.017, 0.016, 0.017, 0.080, 0.082
.......................
815, 20.27, 0.013, 0.014, 0.017, 0.017, 0.016, 0.017, 0.080, 0.083
.......................
816, 20.27, 0.013, 0.014, 0.017, 0.017, 0.016, 0.017, 0.080, 0.083
.......................
817, 20.27, 0.013, 0.014, 0.017, 0.017, 0.016, 0.017, 0.080, 0.082
.......................
818, 20.27, 0.013, 0.014, 0.017, 0.017, 0.016, 0.017, 0.079, 0.083
.......................
819, 20.27, 0.013, 0.014, 0.017, 0.017, 0.016, 0.017, 0.079, 0.083

.......................
989, 20.27, 0.013, 0.014, 0.016, 0.017, 0.016, 0.016, 0.078, 0.082
.......................
990, 20.27, 0.013, 0.014, 0.016, 0.017, 0.016, 0.016, 0.078, 0.081
.......................
991, 20.27, 0.013, 0.014, 0.016, 0.017, 0.016, 0.016, 0.078, 0.081
.......................
992, 20.27, 0.013, 0.014, 0.016, 0.017, 0.016, 0.016, 0.078, 0.081
.......................
993, 20.27, 0.013, 0.014, 0.016, 0.017, 0.016, 0.016, 0.078, 0.082
.......................
994, 20.27, 0.013, 0.014, 0.016, 0.017, 0.016, 0.016, 0.078, 0.081
.......................
995, 20.27, 0.013, 0.014, 0.016, 0.017, 0.016, 0.016, 0.078, 0.081
.......................
996, 20.27, 0.013, 0.014, 0.016, 0.017, 0.016, 0.016, 0.078, 0.081
.......................
997, 20.27, 0.013, 0.014, 0.016, 0.017, 0.016, 0.016, 0.078, 0.082
.......................
998, 20.30, 0.013, 0.014, 0.016, 0.017, 0.016, 0.016, 0.078, 0.082
.......................
999, 20.27, 0.013, 0.014, 0.016, 0.017, 0.016, 0.016, 0.079, 0.081